In [1]:
import numpy as np
import mne

from hdf5storage import loadmat, savemat 

In [2]:
outdict=loadmat('preprocessed_eeg.mat')
outdict.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Fs', 'badchan', 'ch_bad', 'ch_labels', 'chanlocs', 'eyechans', 'goodchan', 'preprocessed_eeg', 'subject_ID'])

In [3]:
preprocessed_eeg=outdict['preprocessed_eeg']*0.000001 # reduce the amplitube to be shown on MNE's plot
sampling_freq=outdict['Fs'][0]
ch_bad=outdict['ch_bad'][0]-1
eyechans=outdict['eyechans'][0]-1
badchan=outdict['badchan'][0]-1
goodchan=outdict['goodchan'][0]-1
subject_ID=outdict['subject_ID'][0]

n_channels=np.shape(preprocessed_eeg)[0]

In [4]:
ch_names = [f"E{n}" for n in range(1, 257)]
ch_types = ["eeg"] * n_channels
info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage("GSN-HydroCel-256",match_case=False,match_alias=False,on_missing='raise')
info["description"] = subject_ID
print(info)

<Info | 9 non-empty values
 bads: []
 ch_names: E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11, E12, E13, E14, ...
 chs: 256 EEG
 custom_ref_applied: False
 description: DAON
 dig: 259 items (3 Cardinal, 256 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 256
 projs: []
 sfreq: 1000.0 Hz
>


In [5]:
ch_info_bads=np.concatenate((ch_bad, eyechans, badchan), axis=0)

In [6]:
ch_info_bads

array([240, 241, 242, 237, 238, 239, 243, 244, 245, 246, 250, 255,  90,
       101, 110, 119, 132, 144, 164, 173, 186, 198, 207, 215, 228, 232,
       236, 235, 234, 233, 231, 227, 216, 208, 199, 187, 174, 165, 155,
       145, 133, 120, 111, 102,  91,  81, 254, 249, 256, 247, 251, 252,
       218, 230, 226, 217, 247, 251, 252,  66, 248, 253,  72,  24,  80,
        88], dtype=uint16)

In [8]:
ch_names[255]

'E256'

In [ ]:
info['bads']

In [10]:
ch_labels_info_bads=list()
for k in range(len(ch_info_bads)):
    ch_labels_info_bads.append(ch_names[k])

info['bads'] = ch_labels_info_bads

In [16]:
raw = mne.io.RawArray(preprocessed_eeg, info)
raw.set_eeg_reference('average', projection=True)
# raw.plot(show_scrollbars=False, show_scalebars=False)

Creating RawArray with float64 data, n_channels=256, n_times=182000
    Range : 0 ... 181999 =      0.000 ...   181.999 secs
Ready.


In [ ]:
raw.save("raw.fif",overwrite=True)